### 2-3번 문제

In [2]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin, urlparse, parse_qs
import math

def download_one_episode(title, no, url):
    dir_path = os.path.join('img', title, str(no))
    os.makedirs(dir_path, exist_ok=True)
    req_header = {
        'referer': url,
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'
    }
    res = requests.get(url, headers=req_header)
    if not res.ok:
        print(f'Error Code = {res.status_code}')
        return
    soup = BeautifulSoup(res.text, 'html.parser')
    img_tags = soup.select("img[src*='IMAG01']")
    img_url_list = [img_tag['src'] for img_tag in img_tags]
    print(f'Found {len(img_url_list)} images in episode {no}.')
    for img_url in img_url_list:
        res_img = requests.get(img_url, headers=req_header)
        if res_img.ok:
            img_data = res_img.content
            file_name = os.path.basename(img_url)
            file_path = os.path.join(dir_path, file_name)
            with open(file_path, 'wb') as file:
                print(f'Writing to {file_path} ({len(img_data):,} bytes)')
                file.write(img_data)
        else:
            print(f'Error Code = {res_img.status_code} for {img_url}')

def download_all_episode(title, episode_url, max_episodes=None):
    # titleId 추출
    parsed = urlparse(episode_url)
    qs = parse_qs(parsed.query)
    title_id = qs.get('titleId', [None])[0]
    if not title_id:
        print('titleId not found in URL.')
        return

    # 1페이지에서 전체 회차 수 확인
    api_url = f'https://comic.naver.com/api/article/list?titleId={title_id}&page=1'
    req_header = {
        'referer': episode_url,
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'
    }
    res = requests.get(api_url, headers=req_header)
    if not res.ok:
        print(f'Error Code = {res.status_code}')
        return
    data = res.json()
    total_count = data.get('totalCount', 0)
    page_size = data.get('pageSize', 10)
    total_page = math.ceil(total_count / page_size) if page_size else 1

    print(f'Total {total_count} episodes, {total_page} pages for {title}.')

    episode_set = set()
    episode_counter = 0
    for page in range(1, 2): #전체 페이지 다 받고싶으시면 2대신 total_page + 1로 변경하세요.
        if max_episodes is not None and episode_counter >= max_episodes:
            break
        api_url = f'https://comic.naver.com/api/article/list?titleId={title_id}&page={page}'
        res = requests.get(api_url, headers=req_header)
        if not res.ok:
            print(f'Error Code = {res.status_code} for page {page}')
            continue
        data = res.json()
        article_list = data.get('articleList', [])
        for article in article_list:
            if max_episodes is not None and episode_counter >= max_episodes:
                break
            no = str(article.get('no'))
            if no and no not in episode_set:
                episode_set.add(no)
                full_url = f'https://comic.naver.com/webtoon/detail?titleId={title_id}&no={no}'
                download_one_episode(title, no, full_url)
                episode_counter += 1


In [3]:
download_all_episode('배달왕', 'https://comic.naver.com/webtoon/list?titleId=823933')


Total 134 episodes, 14 pages for 배달왕.
Found 44 images in episode 130.
Writing to img\배달왕\130\20250710220622_f782a230cd104a88a40917f8272635d4_IMAG01_1.jpg (136,954 bytes)
Writing to img\배달왕\130\20250710220622_f782a230cd104a88a40917f8272635d4_IMAG01_2.jpg (106,303 bytes)
Writing to img\배달왕\130\20250710220622_f782a230cd104a88a40917f8272635d4_IMAG01_3.jpg (142,449 bytes)
Writing to img\배달왕\130\20250710220622_f782a230cd104a88a40917f8272635d4_IMAG01_4.jpg (133,923 bytes)
Writing to img\배달왕\130\20250710220622_f782a230cd104a88a40917f8272635d4_IMAG01_5.jpg (107,165 bytes)
Writing to img\배달왕\130\20250710220622_f782a230cd104a88a40917f8272635d4_IMAG01_6.jpg (136,999 bytes)
Writing to img\배달왕\130\20250710220622_f782a230cd104a88a40917f8272635d4_IMAG01_7.jpg (82,487 bytes)
Writing to img\배달왕\130\20250710220622_f782a230cd104a88a40917f8272635d4_IMAG01_8.jpg (112,916 bytes)
Writing to img\배달왕\130\20250710220622_f782a230cd104a88a40917f8272635d4_IMAG01_9.jpg (106,265 bytes)
Writing to img\배달왕\130\20250710